In [1]:
import pandas as pd
import regex as re
import pygsheets
from datetime import date, timedelta, datetime
import numpy as np
import geopandas as gpd

In [2]:
ASSAM_REVENUE_CIRCLES = pd.read_csv('D:/Projects/assam-tender-scraper/FRIMS_Daily_Reports_Scraper/Data/Assam_Maps/ASSAM_REVENUE_CIRCLES.csv')
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt\)','', regex=True)

In [4]:
ASSAM_REVENUE_CIRCLES = pd.read_csv('Data/Assam_Maps/ASSAM_REVENUE_CIRCLES.csv')
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-I\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-II\)','', regex=True)

In [5]:
google_client = pygsheets.authorize(service_file='idea-frm-500603810b1a.json')
FRIMS_2022_google_sheet = google_client.open('FRIMS_2022')
INFRA_DAMAGES_google_sheet =google_client.open('INFRA DAMAGES PER REVENUE CIRCLE')

# Table of Contents:
* [Population affected per revenue circle](#population)
* [Crop affected per revenue circle](#cropaffected)
* [Relief camps per revenue cirlce](#reliefcamps)
* [Relief distribution centres per revenue circle](#distribution)
* [Relief camp inmates per revenue circle](#inmates)
* [Human lives lost per revenue circle](#liveslost)
* [Roads damaged per revenue circle](#roadsdamaged)
* [Embankment damages per revenue circle](#embankmentdamages)
* [Embankments affected per revenue circle](#embankmentaffected)
* [Bridges damaged per revenue circle](#bridgesdamaged)
* [Other damages per revenue circle](#otherdamages)
* [Erosion damages per revenue circle](#erosion)
* [Wildlife damages per revenue circle](#wildlife)
* [Landslides per revenue circle](#landslides)
* [Urban floods per revenue circle](#urbanfloods)

## POPULATION AFFECTED <a class="anchor" id="population"></a>

In [101]:
population_df = pd.read_csv('Data/Cleaned Data/POPULATION_AND_CROP_AREA_AFFECTED_MASTER_2022.csv')

population_df_revcircle = pd.DataFrame(columns=['District','RevenueCircle','Population affected','Date'])
i=0
for idx, row in population_df.iterrows():
    district = row['District']
    date = row['Date']
    
    population_affected_details = row['Population Details']
    for rev_circle_detail in population_affected_details.split(','):
        rev_circle_detail = rev_circle_detail.lower()
        population_affected = re.findall(r'\d+',rev_circle_detail.split(',')[0])[0]
        rev_circle = re.findall(r'[a-z]+',rev_circle_detail.split(',')[0])[0]
        
        population_df_revcircle.loc[i,'District'] = district
        population_df_revcircle.loc[i,'RevenueCircle'] = rev_circle
        population_df_revcircle.loc[i,'Population affected'] = population_affected 
        population_df_revcircle.loc[i,'Date'] = date
        i=i+1
        
population_df_revcircle['Date'] = pd.to_datetime(population_df_revcircle.Date)

In [102]:
POPULATION_AFFECTED = FRIMS_2022_google_sheet.worksheet('title','POPULATION_AFFECTED')
POPULATION_AFFECTED.clear()
POPULATION_AFFECTED.set_dataframe(population_df_revcircle,(1,1))

## CROP AFFECTED <a class="anchor" id="cropaffected"></a>

In [120]:
crop_df = pd.read_csv('Data/Cleaned Data/POPULATION_AND_CROP_AREA_AFFECTED_MASTER_2022.csv')

crop_df_revcircle = pd.DataFrame(columns=['District','RevenueCircle','Crop Area Affected (Hectares)','Date'])
i = 0
for idx, row in crop_df.iterrows():
    district = row['District']
    date = row['Date']
    
    crop_affected_details = row['Crop Area Details']
    for rev_circle_detail in crop_affected_details.split(','):
        rev_circle_detail = rev_circle_detail.lower()
        area_affected = re.findall(r'\d+\.\d+',rev_circle_detail.split(',')[0])[0]
        rev_circle = re.findall(r'[a-z]+',rev_circle_detail.split(',')[0])[0]
        
        crop_df_revcircle.loc[i,'District'] = district
        crop_df_revcircle.loc[i,'RevenueCircle'] = rev_circle
        crop_df_revcircle.loc[i,'Crop Area Affected (Hectares)'] = area_affected 
        crop_df_revcircle.loc[i,'Date'] = date
        i=i+1
        
crop_df_revcircle['Date'] = pd.to_datetime(crop_df_revcircle.Date)

In [121]:
CROP_AFFECTED = FRIMS_2022_google_sheet.worksheet('title','CROP_AREA_AFFECTED')
CROP_AFFECTED.clear()
CROP_AFFECTED.set_dataframe(crop_df_revcircle,(1,1))

## RELIEF CAMPS <a class="anchor" id="reliefcamps"></a>

In [135]:
camps_df = pd.read_csv('Data/Cleaned Data/RELIEF_CAMPS_OPENED_MASTER_2022.csv')

camps_df_revcircle = pd.DataFrame(columns=['District','RevenueCircle','Number of camps','Date'])
i = 0
for idx, row in camps_df.iterrows():
    district = row['District']
    date = row['Date']
    
    no_camps_details = row['Relief Camp'].split('(')[1]
    for rev_circle_detail in no_camps_details.split(','):
        rev_circle_detail = rev_circle_detail.lower()
        no_camps = re.findall(r'\d+',rev_circle_detail.split(',')[0])[0]
        rev_circle = re.findall(r'[a-z]+',rev_circle_detail.split(',')[0])[0]
        
        camps_df_revcircle.loc[i,'District'] = district
        camps_df_revcircle.loc[i,'RevenueCircle'] = rev_circle
        camps_df_revcircle.loc[i,'Number of camps'] = no_camps 
        camps_df_revcircle.loc[i,'Date'] = date
        i=i+1
        
camps_df_revcircle['Date'] = pd.to_datetime(camps_df_revcircle.Date)

In [136]:
CAMPS_OPENED = FRIMS_2022_google_sheet.worksheet('title','RELIEF_CAMPS_OPENED')
CAMPS_OPENED.clear()
CAMPS_OPENED.set_dataframe(camps_df_revcircle,(1,1))

## RELIEF DISTRIBUTION CENTRES <a class="anchor" id="distribution"></a>

In [424]:
distributioncenters_df = pd.read_csv('Data/Cleaned Data/RELIEF_CAMPS_OPENED_MASTER_2022.csv')

distributioncenters_df_revcircle = pd.DataFrame(columns=['District','RevenueCircle','Number of centers','Date'])
i = 0
for idx, row in camps_df.iterrows():
    district = row['District']
    date = row['Date']
    
    no_camps_details = row['Relief Distribution Centres'].split('(')[1]
    for rev_circle_detail in no_camps_details.split(','):
        rev_circle_detail = rev_circle_detail.lower()
        no_camps = re.findall(r'\d+',rev_circle_detail.split(',')[0])[0]
        rev_circle = re.findall(r'[a-z]+',rev_circle_detail.split(',')[0])[0]
        
        distributioncenters_df_revcircle.loc[i,'District'] = district
        distributioncenters_df_revcircle.loc[i,'RevenueCircle'] = rev_circle
        distributioncenters_df_revcircle.loc[i,'Number of centers'] = no_camps 
        distributioncenters_df_revcircle.loc[i,'Date'] = date
        i=i+1
        
distributioncenters_df_revcircle['Date'] = pd.to_datetime(distributioncenters_df_revcircle.Date)

In [426]:
RELIEF_DISTRIBUTION_CENTRES = FRIMS_2022_google_sheet.worksheet('title','RELIEF_DISTRIBUTION_CENTRES')
RELIEF_DISTRIBUTION_CENTRES.clear()
RELIEF_DISTRIBUTION_CENTRES.set_dataframe(distributioncenters_df_revcircle,(1,1))

## RELIEF CAMP INMATES <a class="anchor" id="inmates"></a>

In [11]:
inmates_df = pd.read_csv('Data/Cleaned Data/RELIEF_CAMP_INMATES_MASTER_2022.csv')

inmates_df_revcircle = pd.DataFrame(columns=['District','RevenueCircle','Number of inmates','Date'])
i = 0
for idx, row in inmates_df.iterrows():
    district = row['District']
    date = row['Date']
    
    inmates_details = row['Revenue Circlewise']
    for rev_circle_detail in inmates_details.split(','):
        rev_circle_detail = rev_circle_detail.lower()
        no_camps = re.findall(r'\d+',rev_circle_detail.split(',')[0])[0]
        rev_circle = re.findall(r'[a-z]+',rev_circle_detail.split(',')[0])[0]
        
        inmates_df_revcircle.loc[i,'District'] = district
        inmates_df_revcircle.loc[i,'RevenueCircle'] = rev_circle.capitalize()
        inmates_df_revcircle.loc[i,'Number of inmates'] = no_camps 
        inmates_df_revcircle.loc[i,'Date'] = date
        i=i+1
        
inmates_df_revcircle['Date'] = pd.to_datetime(inmates_df_revcircle.Date)

In [13]:
INMATES = FRIMS_2022_google_sheet.worksheet('title','RELIEF_CAMP_INMATES')
INMATES.clear()
INMATES.set_dataframe(inmates_df_revcircle,(1,1))

## HUMAN LIVES LOST <a class="anchor" id="liveslost"></a>

In [141]:
liveslost_df = pd.read_csv('Data/Cleaned Data/FRIMS_HUMAN_LIVES_LOST_CONFIRMED_MASTER_2022.csv')

liveslost_df_revcircle = pd.DataFrame(columns=['District','RevenueCircle','Lives lost','Date'])
i=0
for idx, row in liveslost_df.iterrows():
    district = row['District']
    date = row['Date']
    
    liveslost_details = row['Revenue Circlewise']
    
    for rev_circle_detail in liveslost_details.split(','):
        rev_circle_detail = rev_circle_detail.lower()
        
        lives_lost = re.findall(r'\d+',rev_circle_detail.split(',')[0])[0]
        rev_circle = re.findall(r'[a-z]+',rev_circle_detail.split(',')[0])[0]
        liveslost_df_revcircle.loc[i,'District'] = district
        liveslost_df_revcircle.loc[i,'RevenueCircle'] = rev_circle.capitalize()
        liveslost_df_revcircle.loc[i,'Lives lost'] = lives_lost 
        liveslost_df_revcircle.loc[i,'Date'] = date 
        i=i+1
        
liveslost_df_revcircle['Date'] = pd.to_datetime(liveslost_df_revcircle.Date)

In [142]:
LIVESLOST = FRIMS_2022_google_sheet.worksheet('title','HUMAN_LIVES_LOST_CONFIRMED')
LIVESLOST.clear()
LIVESLOST.set_dataframe(liveslost_df_revcircle,(1,1))

## ROADS DAMAGED <a class="anchor" id="roadsdamaged"></a>

In [307]:
def get_revcircle_damages_dict(ROW):    
    DETAIL = ROW['Details']
    REV_CIRCLE_DAMAGES_DICT = dict()
    
    count = 0
    rev_circles_identified = []
    rev_circles_identifed_at = []
    for REV_CIRCLE in REVENUE_CIRCLES:
        if REV_CIRCLE in DETAIL:
            rev_circles_identified.append(REV_CIRCLE)
            rev_circles_identifed_at.append(DETAIL.index(REV_CIRCLE))
            count = count+1
    REV_CIRCLES_IDENTIFIED_DF = pd.DataFrame([rev_circles_identified,rev_circles_identifed_at]).T
    REV_CIRCLES_IDENTIFIED_DF.columns = ['RevenueCircle','StringIndex']
    REV_CIRCLES_IDENTIFIED_DF = REV_CIRCLES_IDENTIFIED_DF.sort_values(by='StringIndex').reset_index(drop=True)
    
    for i,val in enumerate(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
        StringStartIndex = val
        if i+1 == len(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
            StringEndIndex = None
        else:
            StringEndIndex = REV_CIRCLES_IDENTIFIED_DF.StringIndex[i+1]
        
        SUB_DETAIL = DETAIL[StringStartIndex:StringEndIndex]
        
        if other_damages == True:
            NO_DAMAGES = len(re.findall('\|', SUB_DETAIL))
            print(NO_DAMAGES)
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES-1
            continue 
            
        NO_DAMAGES_DateAlgo = len(re.findall('2022',SUB_DETAIL)) + len(re.findall(r'\d+/\d+/\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+-\d+-\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+\.\d+\.\d{2}\s+', SUB_DETAIL))
        # Number of 2022 in the DETAIL +  Number of dd/mm/22 in the DETAIL +  Number of dd-mm-22 in the DETAIL + Number of dd.mm.22 in the DETAIL
        NO_DAMAGES_LongAlgo = len(re.findall('Long -',SUB_DETAIL)) 
    
        if float(ROW['Number of Damages (Long)'])>0:
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        else:
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_DateAlgo
    
    
    return REV_CIRCLE_DAMAGES_DICT    

In [225]:
roads_damaged_df = pd.read_csv('Data/Cleaned Data/FRIMS_ROAD_DAMAGES_MASTER_2022.csv')
roads_damaged_df.shape

(577, 4)

In [226]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in roads_damaged_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = roads_damaged_df[roads_damaged_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Dhemaji
['Dhakuakhana -', 'Subansiri -', 'Dhemaji -', 'Sissiborgaon -', 'Jonai -', 'Gogamukh -']
----
Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/2785225505.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


Hailakandi
['Ramkrishna Nagar -', 'Algapur -', 'Lala -', 'Katlicherra -']
----
Nagaon
['Kaliabor -', 'Samaguri -', 'Rupahi -', 'Dhing -', 'Nagaon -', 'Raha -', 'Kampur -']
----
Barpeta
['Kalgachia -', 'Baghbor -', 'Barpeta -', 'Sarthebari -', 'Barnagar -', 'Sarupeta -', 'Chenga -']
----
Hojai
['Hojai -', 'Doboka -', 'Lanka -']
----
Kamrup
['Rangia -', 'Kayan -', 'Kamalpur -', 'Hajo -', 'Chhaygaon -', 'Goroimari -', 'Chamaria -', 'Nagarbera -', 'Boko -', 'North\nGuwahati -', 'Jalah -', 'Palasbari -', 'Azara -']
----
Lakhimpur
['Narayanpur -', 'Bihpuria -', 'Nowboicha -', 'Kadam -', 'North Lakhimpur -', 'Dhakuakhana -', 'Subansiri -']
----
Udalguri
['Dhekiajuli -', 'Khoirabari -', 'Pathorighat -', 'Mangaldoi -', 'Dalgaon -', 'Harisinga -', 'Udalguri -', 'Mazbat -', 'Kalaigaon -']
----
Tamulpur
['Rangia -', 'Pathorighat -', 'Bajali -', 'Goreswar -', 'Tamulpur -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Bongaigaon
['Bongaigaon -', 'B

In [223]:
FILTERED_DAMAGES_dfs[0]

,District,Number,Details,Date,Detail_Length,Number of Damages (Date),Number of Damages (Long),NumberofDamages_RevenueCircle_level
470,Sivasagar,1,Sivsagar - Damaged of Road cum Embankment of D...,06-07-2022,285,1,1,{'Sivsagar ': 1}
484,Sivasagar,1,Demow - Approach road to Lakhmoni Ghaat | Poth...,08-07-2022,305,2,1,{'Demow ': 1}
550,Sivasagar,1,Nazira - At some locations Duwarah Gaon Road g...,26-08-2022,360,1,1,{'Nazira ': 1}
487,Sivasagar,4,Nazira - Dhudor Ali to Rohdoi Ali | Mohan Gaon...,09-07-2022,722,4,4,{'Nazira ': 4}


In [228]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

ROAD_DAMAGES = FRIMS_2022_google_sheet.worksheet('title','ROADS_DAMAGED')
ROAD_DAMAGES.clear()
ROAD_DAMAGES.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                           (1,1))

In [229]:
road_damages_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        road_damages_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        road_damages_revcircle_df.loc[i,'District'] = row['District']
        road_damages_revcircle_df.loc[i,'Number of damages'] = no_damages
        road_damages_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
road_damages_revcircle_df.Date = pd.to_datetime(road_damages_revcircle_df.Date)

In [230]:
ROAD_DAMAGES = INFRA_DAMAGES_google_sheet.worksheet('title','ROADS_DAMAGED')
ROAD_DAMAGES.clear()
ROAD_DAMAGES.set_dataframe(road_damages_revcircle_df,
                           (1,1))

## EMBANKMENT BREACHES <a class="anchor" id="embankmentbreaches"></a>

In [247]:
embankments_breached_df = pd.read_csv('Data/Cleaned Data/FRIMS_DAMAGE_EMBANKMENT_BREACHED_MASTER_2022.csv')
embankments_breached_df.shape

(93, 4)

In [248]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_breached_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_breached_df[embankments_breached_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Hojai
['Hojai -', 'Doboka -', 'Lanka -']
----
Tamulpur
['Rangia -', 'Pathorighat -', 'Bajali -', 'Goreswar -', 'Tamulpur -']
----
Biswanath
['Biswanath -', 'Halem -', 'Gohpur -']
----
Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----
Udalguri
['Dhekiajuli -', 'Khoirabari -', 'Pathorighat -', 'Mangaldoi -', 'Dalgaon -', 'Harisinga -', 'Udalguri -', 'Mazbat -', 'Kalaigaon -']
----
Sonitpur
['Dhekiajuli -', 'Chariduar -', 'Tezpur -', 'Na-Duar -']
----


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/3615585965.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


Nalbari
['Tihu -', 'Pachim\nNalbari -', 'Barkhetri -', 'Barbhag -', 'Nalbari -', 'Banekuchi -', 'Ghograpar -', 'Ghograpar -', 'Baganpara -']
----
Lakhimpur
['Narayanpur -', 'Bihpuria -', 'Nowboicha -', 'Kadam -', 'North Lakhimpur -', 'Dhakuakhana -', 'Subansiri -']
----
Goalpara
['Lakhipur -', 'Balijana -', 'Matia -', 'Dudhnoi -', 'Rangjuli -']
----
Barpeta
['Kalgachia -', 'Baghbor -', 'Barpeta -', 'Sarthebari -', 'Barnagar -', 'Sarupeta -', 'Chenga -']
----
Nagaon
['Kaliabor -', 'Samaguri -', 'Rupahi -', 'Dhing -', 'Nagaon -', 'Raha -', 'Kampur -']
----
Morigaon
['Bhuragaon -', 'Laharighat -', 'Morigaon -', 'Mikirbheta -', 'Mayong -']
----
Cachar
['Katigorah -', 'Silchar -', 'Udharbond -', 'Sonai -', 'Lakhipur -']
----
Bajali
['Bajali -', 'Jalah -']
----
Baksa
['Barnagar -', 'Ghograpar -', 'Jalah -', 'Barama -', 'Baganpara -', 'Baska -', 'Sarupeta -']
----
Kamrup
['Rangia -', 'Kayan -', 'Kamalpur -', 'Hajo -', 'Chhaygaon -', 'Goroimari -', 'Chamaria -', 'Nagarbera -', 'Boko -', 'North

In [249]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

EMBANKMENT_BREACHES = FRIMS_2022_google_sheet.worksheet('title','EMBANKMENT_BREACHES')
EMBANKMENT_BREACHES.clear()
EMBANKMENT_BREACHES.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                           (1,1))

In [250]:
embankment_breaches_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_breaches_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_breaches_revcircle_df.loc[i,'District'] = row['District']
        embankment_breaches_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_breaches_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_breaches_revcircle_df.Date = pd.to_datetime(embankment_breaches_revcircle_df.Date)

In [251]:
EMBANKMENT_BREACHES = INFRA_DAMAGES_google_sheet.worksheet('title','EMBANKMENT_BREACHES')
EMBANKMENT_BREACHES.clear()
EMBANKMENT_BREACHES.set_dataframe(embankment_breaches_revcircle_df,
                           (1,1))

## EMBANKMENT AFFECTED <a class="anchor" id="embankmentaffected"></a>

In [255]:
embankments_affected_df = pd.read_csv('Data/Cleaned Data/FRIMS_DAMAGE_EMBANKMENT_AFFECTED_MASTER_2022.csv')
embankments_affected_df.shape

(131, 4)

In [256]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_affected_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_affected_df[embankments_affected_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Jorhat
['Jorhat West -', 'Jorhat East -', 'Teok -', 'Sarupathar -', 'Mariani -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Lakhimpur
['Narayanpur -', 'Bihpuria -', 'Nowboicha -', 'Kadam -', 'North Lakhimpur -', 'Dhakuakhana -', 'Subansiri -']
----
Nagaon
['Kaliabor -', 'Samaguri -', 'Rupahi -', 'Dhing -', 'Nagaon -', 'Raha -', 'Kampur -']
----
Dhemaji
['Dhakuakhana -', 'Subansiri -', 'Dhemaji -', 'Sissiborgaon -', 'Jonai -', 'Gogamukh -']
----
Majuli
['Majuli -', 'Ujani Majuli -']
----
Nalbari


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/369729252.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


['Tihu -', 'Pachim\nNalbari -', 'Barkhetri -', 'Barbhag -', 'Nalbari -', 'Banekuchi -', 'Ghograpar -', 'Ghograpar -', 'Baganpara -']
----
Tamulpur
['Rangia -', 'Pathorighat -', 'Bajali -', 'Goreswar -', 'Tamulpur -']
----
Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----
Karbi Anglong West
['Donkamokam -']
----
Dibrugarh
['Dibrugarh West -', 'Dibrugarh East -', 'Chabua -', 'Tengakhat -', 'Moran -', 'Tingkhong -', 'Naharkatia -']
----
Udalguri
['Dhekiajuli -', 'Khoirabari -', 'Pathorighat -', 'Mangaldoi -', 'Dalgaon -', 'Harisinga -', 'Udalguri -', 'Mazbat -', 'Kalaigaon -']
----
Cachar
['Katigorah -', 'Silchar -', 'Udharbond -', 'Sonai -', 'Lakhipur -']
----
Bajali
['Bajali -', 'Jalah -']
----
Baksa
['Barnagar -', 'Ghograpar -', 'Jalah -', 'Barama -', 'Baganpara -', 'Baska -', 'Sarupeta -']
----
Barpeta
['Kalgachia -', 'Baghbor -', 'Barpeta -', 'Sarthebari -', 'Barnagar -', 'Sarupeta -', 'Chenga -']
----
South Salmara
['South Salmara -', 'Mankacha

In [257]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

EMBANKMENT_BREACHES = FRIMS_2022_google_sheet.worksheet('title','EMBANKMENT_AFFECTED')
EMBANKMENT_BREACHES.clear()
EMBANKMENT_BREACHES.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                           (1,1))

In [259]:
embankment_affected_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_affected_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_affected_revcircle_df.loc[i,'District'] = row['District']
        embankment_affected_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_affected_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_affected_revcircle_df.Date = pd.to_datetime(embankment_affected_revcircle_df.Date)

In [260]:
EMBANKMENT_AFFECTED = INFRA_DAMAGES_google_sheet.worksheet('title','EMBANKMENT_AFFECTED')
EMBANKMENT_AFFECTED.clear()
EMBANKMENT_AFFECTED.set_dataframe(embankment_affected_revcircle_df,
                           (1,1))

## BRIDGES DAMAGED <a class="anchor" id="bridgesdamaged"></a>

In [261]:
bridges_df = pd.read_csv('Data/Cleaned Data/FRIMS_BRIDGES_DAMAGED_MASTER_2022.csv')
bridges_df.shape

(106, 4)

In [262]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in bridges_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = bridges_df[bridges_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Tamulpur
['Rangia -', 'Pathorighat -', 'Bajali -', 'Goreswar -', 'Tamulpur -']
----
Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----
Nalbari
['Tihu -', 'Pachim\nNalbari -', 'Barkhetri -', 'Barbhag -', 'Nalbari -', 'Banekuchi -', 'Ghograpar -', 'Ghograpar -', 'Baganpara -']
----
Cachar
['Katigorah -', 'Silchar -', 'Udharbond -', 'Sonai -', 'Lakhipur -']
----
Baksa
['Barnagar -', 'Ghograpar -', 'Jalah -', 'Barama -', 'Baganpara -', 'Baska -', 'Sarupeta -']
----
Udalguri
['Dhekiajuli -', 'Khoirabari -', 'Pathorighat -', 'Mangaldoi -', 'Dalgaon -', 'Harisinga -', 'Udalguri -', 'Mazbat -', 'Kalaigaon -']
----
Sonitpur
['Dhekiajuli -', 'Chariduar -', 'Tezpur -', 'Na-Duar -']
----
Biswanath


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/1038643447.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


['Biswanath -', 'Halem -', 'Gohpur -']
----
Barpeta
['Kalgachia -', 'Baghbor -', 'Barpeta -', 'Sarthebari -', 'Barnagar -', 'Sarupeta -', 'Chenga -']
----
Nagaon
['Kaliabor -', 'Samaguri -', 'Rupahi -', 'Dhing -', 'Nagaon -', 'Raha -', 'Kampur -']
----
Hailakandi
['Ramkrishna Nagar -', 'Algapur -', 'Lala -', 'Katlicherra -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Dima-Hasao
['Umrangso -', 'Haflong -', 'Mahur -', 'Maibong -']
----
Goalpara
['Lakhipur -', 'Balijana -', 'Matia -', 'Dudhnoi -', 'Rangjuli -']
----
Kamrup
['Rangia -', 'Kayan -', 'Kamalpur -', 'Hajo -', 'Chhaygaon -', 'Goroimari -', 'Chamaria -', 'Nagarbera -', 'Boko -', 'North\nGuwahati -', 'Jalah -', 'Palasbari -', 'Azara -']
----
Kokrajhar
['Gossaigaon -', 'Bhawraguri -', 'Dotoma -', 'Kokrajhar -', 'Bagribari -', 'Golokganj -', 'Dhubri -', 'Chapar -']
----
Dhemaji
['Dhakuakhana -', 'Subansiri -', 'Dhemaji -', 'Sissiborgaon -', 'Jonai -', 'Gogamukh -']
----
Morigaon


In [263]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

BRIDGES = FRIMS_2022_google_sheet.worksheet('title','BRIDGES_DAMAGED')
BRIDGES.clear()
BRIDGES.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                           (1,1))

In [264]:
bridges_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        bridges_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        bridges_revcircle_df.loc[i,'District'] = row['District']
        bridges_revcircle_df.loc[i,'Number of damages'] = no_damages
        bridges_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
bridges_revcircle_df.Date = pd.to_datetime(bridges_revcircle_df.Date)

In [265]:
BRIDGES = INFRA_DAMAGES_google_sheet.worksheet('title','BRIDGES_DAMAGED')
BRIDGES.clear()
BRIDGES.set_dataframe(bridges_revcircle_df,
                           (1,1))

## OTHER DAMAGES <a class="anchor" id="otherdamages"></a>

In [270]:
others_df = pd.read_csv('Data/Cleaned Data/FRIMS_OTHER_DAMAGES_MASTER_2022.csv')
others_df.tail()

,District,Details,Date
721,Udalguri,Udalguri - Irrigation Canal/Scheme Damaged | A...,14-10-2022
722,Lakhimpur,Bihpuria - Fisheries/Pond | 50 nos. of fishery...,15-10-2022
723,Biswanath,Biswanath - Fisheries/Pond | As per report rec...,15-10-2022
724,Kokrajhar,Gossaigaon - Bamboo Foot Bridge | AS REPORTED ...,16-10-2022
725,Morigaon,Bhuragaon - Elementary Schools Damaged | Borha...,20-10-2022


In [274]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in others_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = others_df[others_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Goalpara
['Lakhipur -', 'Balijana -', 'Matia -', 'Dudhnoi -', 'Rangjuli -']
----
Nalbari
['Tihu -', 'Pachim\nNalbari -', 'Barkhetri -', 'Barbhag -', 'Nalbari -', 'Banekuchi -', 'Ghograpar -', 'Ghograpar -', 'Baganpara -']
----
Nagaon
['Kaliabor -', 'Samaguri -', 'Rupahi -', 'Dhing -', 'Nagaon -', 'Raha -', 'Kampur -']
----


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/1360145020.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----
Hojai
['Hojai -', 'Doboka -', 'Lanka -']
----
Dibrugarh
['Dibrugarh West -', 'Dibrugarh East -', 'Chabua -', 'Tengakhat -', 'Moran -', 'Tingkhong -', 'Naharkatia -']
----
Lakhimpur
['Narayanpur -', 'Bihpuria -', 'Nowboicha -', 'Kadam -', 'North Lakhimpur -', 'Dhakuakhana -', 'Subansiri -']
----
Sonitpur
['Dhekiajuli -', 'Chariduar -', 'Tezpur -', 'Na-Duar -']
----
Kamrup
['Rangia -', 'Kayan -', 'Kamalpur -', 'Hajo -', 'Chhaygaon -', 'Goroimari -', 'Chamaria -', 'Nagarbera -', 'Boko -', 'North\nGuwahati -', 'Jalah -', 'Palasbari -', 'Azara -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Cachar
['Katigorah -', 'Silchar -', 'Udharbond -', 'Sonai -', 'Lakhipur -']
----
Biswanath
['Biswanath -', 'Halem -', 'Gohpur -']
----
Baksa
['Barnagar -', 'Ghograpar -', 'Jalah -', 'Barama -', 'Baganpara -', 'Baska -', 'Sarupeta -']
----
Tamulpur
['Rangia -', 'Pat

In [277]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

OTHERS = FRIMS_2022_google_sheet.worksheet('title','OTHER_DAMAGES')
OTHERS.clear()
OTHERS.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                          (1,1))

In [278]:
others_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        others_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        others_revcircle_df.loc[i,'District'] = row['District']
        others_revcircle_df.loc[i,'Number of damages'] = no_damages
        others_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
others_revcircle_df.Date = pd.to_datetime(others_revcircle_df.Date)

In [279]:
OTHER_DAMAGES = INFRA_DAMAGES_google_sheet.worksheet('title','OTHER_DAMAGES')
OTHER_DAMAGES.clear()
OTHER_DAMAGES.set_dataframe(others_revcircle_df,
                           (1,1))

## EROSION <a class="anchor" id="erosion"></a>

In [382]:
def get_revcircle_damages_dict(ROW):    
    DETAIL = ROW['Details']
    REV_CIRCLE_DAMAGES_DICT = dict()
    
    count = 0
    rev_circles_identified = []
    rev_circles_identifed_at = []
    for REV_CIRCLE in REVENUE_CIRCLES:
        if REV_CIRCLE in DETAIL:
            rev_circles_identified.append(REV_CIRCLE)
            rev_circles_identifed_at.append(DETAIL.index(REV_CIRCLE))
            count = count+1
    REV_CIRCLES_IDENTIFIED_DF = pd.DataFrame([rev_circles_identified,rev_circles_identifed_at]).T
    REV_CIRCLES_IDENTIFIED_DF.columns = ['RevenueCircle','StringIndex']
    REV_CIRCLES_IDENTIFIED_DF = REV_CIRCLES_IDENTIFIED_DF.sort_values(by='StringIndex').reset_index(drop=True)
    
    for i,val in enumerate(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
        StringStartIndex = val
        if i+1 == len(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
            StringEndIndex = None
        else:
            StringEndIndex = REV_CIRCLES_IDENTIFIED_DF.StringIndex[i+1]
        
        SUB_DETAIL = DETAIL[StringStartIndex:StringEndIndex]
        
         
        NO_DAMAGES = len(re.findall(r'\d{1}\.',SUB_DETAIL.split('|')[0])) + len(SUB_DETAIL.split('|')[0].split(','))
        
        REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES
              
    return REV_CIRCLE_DAMAGES_DICT  

In [383]:
erosion_df = pd.read_csv('Data/Cleaned Data/FRIMS_EROSION_MASTER_2022.csv')
erosion_df.tail()

,District,Details,Date
927,Dibrugarh,Dibrugarh East - PHILONUGURI GRANT 4/152 NO OR...,18-10-2022
928,Morigaon,Laharighat - Leruamukh | As per report of the ...,19-10-2022
929,Dibrugarh,Dibrugarh East - Philonuguri Grant 4/152 No OR...,19-10-2022
930,Lakhimpur,Bihpuria - Puroni Bahgorah (Dhunaguri Embankme...,19-10-2022
931,Morigaon,Bhuragaon - Borhallowkanda | Bhrahmaputra rive...,20-10-2022


In [384]:
global REVENUE_CIRCLES

FILTERED_DAMAGES_dfs = []

for DISTRICT in erosion_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = erosion_df[erosion_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Goalpara
['Lakhipur -', 'Balijana -', 'Matia -', 'Dudhnoi -', 'Rangjuli -']
----
Tinsukia
['Sadiya -', 'Doom Dooma -', 'Tinsukia -', 'Margherita -']
----


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/4177303861.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


Nagaon
['Kaliabor -', 'Samaguri -', 'Rupahi -', 'Dhing -', 'Nagaon -', 'Raha -', 'Kampur -']
----
Nalbari
['Tihu -', 'Pachim\nNalbari -', 'Barkhetri -', 'Barbhag -', 'Nalbari -', 'Banekuchi -', 'Ghograpar -', 'Ghograpar -', 'Baganpara -']
----
Sivasagar
['Demow -', 'Sivsagar -', 'Amguri -', 'Nazira -']
----
Udalguri
['Dhekiajuli -', 'Khoirabari -', 'Pathorighat -', 'Mangaldoi -', 'Dalgaon -', 'Harisinga -', 'Udalguri -', 'Mazbat -', 'Kalaigaon -']
----
Tamulpur
['Rangia -', 'Pathorighat -', 'Bajali -', 'Goreswar -', 'Tamulpur -']
----
Sonitpur
['Dhekiajuli -', 'Chariduar -', 'Tezpur -', 'Na-Duar -']
----
Majuli
['Majuli -', 'Ujani Majuli -']
----
Hailakandi
['Ramkrishna Nagar -', 'Algapur -', 'Lala -', 'Katlicherra -']
----
Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----
Barpeta
['Kalgachia -', 'Baghbor -', 'Barpeta -', 'Sarthebari -', 'Barnagar -', 'Sarupeta -', 'Chenga -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkan

In [385]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

EROSION = FRIMS_2022_google_sheet.worksheet('title','EROSION')
EROSION.clear()
EROSION.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                          (1,1))

In [386]:
erosion_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        erosion_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        erosion_revcircle_df.loc[i,'District'] = row['District']
        erosion_revcircle_df.loc[i,'Number of damages'] = no_damages
        erosion_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
erosion_revcircle_df.Date = pd.to_datetime(erosion_revcircle_df.Date)

In [387]:
EROSION = INFRA_DAMAGES_google_sheet.worksheet('title','EROSION')
EROSION.clear()
EROSION.set_dataframe(erosion_revcircle_df,
                           (1,1))

## WILDLIFE <a class="anchor" id="wildlife"></a>

In [420]:
wildlife_df = pd.read_csv('Data/Cleaned Data/FRIMS_WILDLIFE_MASTER_2022.csv')
wildlife_df.tail()

,District,Wildlife affected under protected areas description,Date
26,Morigaon,Mayong - Pobitora WLS 10nos of camp inundated ...,03-07-2022
27,Morigaon,Mayong - Pobitora WLS 9 number of camps affect...,04-07-2022
28,Morigaon,Mayong - Pobitora WLS 9nos of camps affected a...,05-07-2022
29,Morigaon,Mayong - Pobitora WLS 8 number of camps affect...,06-07-2022
30,Morigaon,Mayong - Pobitora WLS 4nos of camps inundated ...,07-07-2022


In [421]:
WILDLIFE_AFFECTED = FRIMS_2022_google_sheet.worksheet('title','WILDLIFE_AFFECTED')
WILDLIFE_AFFECTED.clear()
WILDLIFE_AFFECTED.set_dataframe(wildlife_df,
                          (1,1))

## LANDSLIDES <a class="anchor" id="landslides"></a>

In [324]:
landslide_df = pd.read_csv('Data/Cleaned Data/FRIMS_LANDSLIDE_MASTER_2022.csv')

In [388]:
global REVENUE_CIRCLES

FILTERED_DAMAGES_dfs = []

for DISTRICT in landslide_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = landslide_df[landslide_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Date'],keep='last')
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Hailakandi
['Ramkrishna Nagar -', 'Algapur -', 'Lala -', 'Katlicherra -']
----
Dima-Hasao
['Umrangso -', 'Haflong -', 'Mahur -', 'Maibong -']
----
Kamrup (M)
['Guwahati -', 'Sonapur -', 'Chandrapur -', 'North Guwahati -']
----
Goalpara
['Lakhipur -', 'Balijana -', 'Matia -', 'Dudhnoi -', 'Rangjuli -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Cachar


C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/928607434.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


['Katigorah -', 'Silchar -', 'Udharbond -', 'Sonai -', 'Lakhipur -']
----
Hojai
['Hojai -', 'Doboka -', 'Lanka -']
----
Kamrup
['Rangia -', 'Kayan -', 'Kamalpur -', 'Hajo -', 'Chhaygaon -', 'Goroimari -', 'Chamaria -', 'Nagarbera -', 'Boko -', 'North\nGuwahati -', 'Jalah -', 'Palasbari -', 'Azara -']
----
Morigaon
['Bhuragaon -', 'Laharighat -', 'Morigaon -', 'Mikirbheta -', 'Mayong -']
----
South Salmara
['South Salmara -', 'Mankachar -']
----
Lakhimpur
['Narayanpur -', 'Bihpuria -', 'Nowboicha -', 'Kadam -', 'North Lakhimpur -', 'Dhakuakhana -', 'Subansiri -']
----
Sonitpur
['Dhekiajuli -', 'Chariduar -', 'Tezpur -', 'Na-Duar -']
----
Dibrugarh
['Dibrugarh West -', 'Dibrugarh East -', 'Chabua -', 'Tengakhat -', 'Moran -', 'Tingkhong -', 'Naharkatia -']
----
Karbi Anglong West
['Donkamokam -']
----


In [389]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)

LANDSLIDES = FRIMS_2022_google_sheet.worksheet('title','LANDSLIDES')
LANDSLIDES.clear()
LANDSLIDES.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                          (1,1))

In [390]:
landslide_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        landslide_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        landslide_revcircle_df.loc[i,'District'] = row['District']
        landslide_revcircle_df.loc[i,'Number of damages'] = no_damages
        landslide_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
landslide_revcircle_df.Date = pd.to_datetime(landslide_revcircle_df.Date)

In [391]:
LANDSLIDE = INFRA_DAMAGES_google_sheet.worksheet('title','LANDSLIDE')
LANDSLIDE.clear()
LANDSLIDE.set_dataframe(landslide_revcircle_df,
                           (1,1))

## URBANFLOODS <a class="anchor" id="urbanfloods"></a>

In [392]:
urbanflood_df = pd.read_csv('Data/Cleaned Data/FRIMS_URBANFLOOD_MASTER_2022.csv')
urbanflood_df

,District,Details,Date
0,Nalbari,Nalbari - Nalbari rown | Nalbari Palla PWD roa...,15-05-2022
1,Kamrup (M),"Dispur - Hatigaon, Keraikuchi, Wireless, Anil ...",15-05-2022
2,Kamrup (M),Sonapur - 12th Mile Ambher Village | Due to he...,17-05-2022
3,Darrang,Dalgaon - Kharupetia Town | Due to havy rain w...,17-05-2022
4,Nalbari,"Nalbari - Palla Road(Ward No-11), Collage road...",18-05-2022
...,...,...,...
94,Kamrup (M),"Dispur - Beharbari, Keraikuchi, Hatigaon, Rukm...",07-09-2022
95,Kamrup (M),"Dispur - Khanapara, Rukminigaon, GS Road, Beha...",13-09-2022
96,Jorhat,Jorhat East - Parijat Nagar (13 No. Block) | W...,20-09-2022
97,Kamrup (M),"Dispur - Hatigaon, Keraikuchi, Rukminigaon | W...",03-10-2022


In [393]:
global REVENUE_CIRCLES

FILTERED_DAMAGES_dfs = []

for DISTRICT in urbanflood_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = urbanflood_df[urbanflood_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Date'],keep='last')
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Nalbari
['Tihu -', 'Pachim\nNalbari -', 'Barkhetri -', 'Barbhag -', 'Nalbari -', 'Banekuchi -', 'Ghograpar -', 'Ghograpar -', 'Baganpara -']
----
Kamrup (M)
['Guwahati -', 'Sonapur -', 'Chandrapur -', 'North Guwahati -']
----
Darrang
['Khoirabari -', 'Patharighat -', 'Sipajhar -', 'Mangaldoi -', 'Dalgaon -']
----
Jorhat
['Jorhat West -', 'Jorhat East -', 'Teok -', 'Sarupathar -', 'Mariani -']
----
Udalguri
['Dhekiajuli -', 'Khoirabari -', 'Pathorighat -', 'Mangaldoi -', 'Dalgaon -', 'Harisinga -', 'Udalguri -', 'Mazbat -', 'Kalaigaon -']
----
Kamrup
['Rangia -', 'Kayan -', 'Kamalpur -', 'Hajo -', 'Chhaygaon -', 'Goroimari -', 'Chamaria -', 'Nagarbera -', 'Boko -', 'North\nGuwahati -', 'Jalah -', 'Palasbari -', 'Azara -']

C:\Users\saikr\AppData\Local\Temp/ipykernel_17480/837238857.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))



----
Morigaon
['Bhuragaon -', 'Laharighat -', 'Morigaon -', 'Mikirbheta -', 'Mayong -']
----
Kokrajhar
['Gossaigaon -', 'Bhawraguri -', 'Dotoma -', 'Kokrajhar -', 'Bagribari -', 'Golokganj -', 'Dhubri -', 'Chapar -']
----
Goalpara
['Lakhipur -', 'Balijana -', 'Matia -', 'Dudhnoi -', 'Rangjuli -']
----
Bajali
['Bajali -', 'Jalah -']
----
Barpeta
['Kalgachia -', 'Baghbor -', 'Barpeta -', 'Sarthebari -', 'Barnagar -', 'Sarupeta -', 'Chenga -']
----
Hailakandi
['Ramkrishna Nagar -', 'Algapur -', 'Lala -', 'Katlicherra -']
----
Karimganj
['Karimganj -', 'Badarpur -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Cachar
['Katigorah -', 'Silchar -', 'Udharbond -', 'Sonai -', 'Lakhipur -']
----
Dibrugarh
['Dibrugarh West -', 'Dibrugarh East -', 'Chabua -', 'Tengakhat -', 'Moran -', 'Tingkhong -', 'Naharkatia -']
----
Tinsukia
['Sadiya -', 'Doom Dooma -', 'Tinsukia -', 'Margherita -']
----


In [394]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
URBANFLOODS = FRIMS_2022_google_sheet.worksheet('title','URBANFLOODS')
URBANFLOODS.clear()
URBANFLOODS.set_dataframe(MASTER_DAMAGES.drop('Detail_Length',axis=1),
                          (1,1))

In [395]:
urbanflood_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        urbanflood_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        urbanflood_revcircle_df.loc[i,'District'] = row['District']
        urbanflood_revcircle_df.loc[i,'Number of damages'] = no_damages
        urbanflood_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
urbanflood_revcircle_df.Date = pd.to_datetime(urbanflood_revcircle_df.Date)

In [396]:
URBANFLOODS = INFRA_DAMAGES_google_sheet.worksheet('title','URBANFLOODS')
URBANFLOODS.clear()
URBANFLOODS.set_dataframe(urbanflood_revcircle_df,
                           (1,1))